# Goals in cleaning up the data.
1. Extract GPS coordinates from Location column (street address)
2. Consolidate number of Incidents (group similar e.g Theft from Motor Vehicle vs. Theft)


In [1]:
import pandas as pd
import numpy as np
import re

In [267]:
# import data
df = pd.read_json('crimeLogData.json')
df.shape

(11240, 7)

In [229]:
# see columns
df.columns

Index(['Comments', 'Disposition', 'Incident', 'Location', 'Occured',
       'Reported', 'UCPD_ID'],
      dtype='object')

In [272]:
# (ignore the fact that Occurred is misspelled smh)
df.rename(columns={'Occured':'Occurred'}, inplace=True)
df.columns

Index(['Comments', 'Disposition', 'Incident', 'Location', 'Occurred',
       'Reported', 'UCPD_ID'],
      dtype='object')

In [5]:
# peek inside
df.head()

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID
0,Woman reports losing a Canon Power Shot digita...,Closed,Lost Property,5810 S. University (Quad),6/28/10 2:45 PM,7/1/10 12:42 PM,W0731
1,Unknown person smashed windshield of a rental ...,Open,Criminal Damage to Vehicle,61st & Drexel,6/29/10 to 7/1/10 3:00 PM to 1:50 PM,7/1/10 2:01 PM,W0732
2,Unknown person smashed rear passenger's side w...,Open,Theft from Motor Vehicle,60th between Ingleside & Ellis,6/29/10 to 7/1/10 5:30 PM to 12:30 PM,7/1/10 2:48 PM,W0733
3,Bicycle taken from bike rack,Open,Theft,60th and Ellis,6/23/10 to 6/29/10 9:00 AM to 5:00 PM,7/1/10 3:13 PM,W0734
4,"Bottle slipped from man's hands, broke causing...",Closed,Chemical Spill,924 E. 57th St. (BSLC),7/1/10 6:15 PM,7/1/10 6:15 PM,W0735


In [6]:
# remove bad data
'''
NaN corresponds to "No Incident Reports" & missing data, and
voids to some internal error
'''
df[9010:9020]

,Comments,Disposition,Incident,Location,Occurred,Reported,UCPD_ID
9010,"Individual reports losing iPhone, possibly on ...",Closed,Lost Property,5640 S. Ellis to 56th & Cornell,3/17/17 10:00 PM to 11:00 PM,3/18/17 10:55 AM,17-00215
9011,Individual collapsed during a track meet / Tra...,Closed,Medical Call,5550 S. University (Crown Field House),3/18/17 12:45 PM,3/18/17 12:45 PM,17-00216
9012,3 unknown suspects struck victim with fists an...,CPD,Information / Robbery / Vehicle Hijacking,5558 S. Kimbark,3/18/17 4:05 PM,3/18/17 4:07 PM,17-00217
9013,Number generated in error,Void,Void,Void,Void,Void,17-00218
9014,NaN,NaN,:,NaN,NaN,NaN,NaN
9015,NaN,NaN,No Incident Reports this date 3/19/17,NaN,NaN,NaN,NaN
9016,NaN,NaN,:,NaN,NaN,NaN,NaN
9017,11 aluminum support beams taken from bleachers...,Open,Theft,821 E. 55th St. (Stagg Field),3/17/17 to 3/20/17 2:00 PM to 10:30 AM,3/20/17 11:05 AM,17-00219
9018,U.S. passport left unattended on copy machine ...,Open,Theft,1009 E. 57th St. (Hitchcock RH),2/14/17 9:50 AM,3/20/17 12:26 PM,17-00220
9019,An unauthorized individual sleeping in the bui...,Ex. Cleared,Assault,6019 S. Ingleside (Campus Housing),3/20/17 7:50 AM,3/20/17 3:20 PM,17-00221


In [273]:
# remove nan values
df = df.dropna()

# remove voids (check for error in comments column)
void_err = lambda x: re.search(r'error', x) is None
df = df[df['Comments'].apply(void_err)]
df.shape

(10075, 7)

## Extract GPS coordinate
We use the geopy, a geocoding web service, module to interact with Google Maps API.
We have addresses that need geolocation coordinates.

In [8]:
from geopy.geocoders import GoogleV3
from geopy.exc import GeocoderTimedOut

In [9]:
# get api-key (unique to user, get a key in the link below)
# https://developers.google.com/maps/documentation/geocoding/start
with open('google-maps-api/api-key.txt', 'r') as f:
    key = f.readline()

# setup geolocator using Google Maps API
geolocator = GoogleV3(api_key=key, user_agent="uchicago_surronding_area", format_string="%s, Chicago, IL")

In [10]:
'''
Function takes an address as a string and formats to improve 
likelihood of getting gps coordinates from geocode.

Returns:
    If location found: a tuple containing the latitude & longitude
    If not: an integer code (user defined)
'''
def address2coordinates(address):
    
    # adding "avenue" helps narrow down location
    def addAvenue(address):
        avenues = 'cottage grove,drexel,ingleside,ellis,greenwood,university,woodlawn,kimbark,\
kenwood,dorchester,blackstone,harper,lake park,stony island,cornell,everett'.split(',')
        
        for ave in avenues:
            if re.search(ave, address):
                address += ' avenue'
                break
                
        return address

    address = re.sub(r' \(.*\)', '', address.lower())  # remove parenthetical info
    address = re.sub(r' at|and ', ' & ', address)      # fix intersection
    
    try:
        # check if address is between streets
        # e.g. (52nd St. between Greenwood & University)
        pattern = re.findall(r'(\w.*(?= between)) between ((?<=between )\w.*(?= &)) & ((?<=\& )\w.*)', address)[0]
        street_1 = '{} {}'.format(pattern[0], pattern[1])
        street_2 = '{} {}'.format(pattern[0], pattern[2])
        location_1 = geolocator.geocode(addAvenue(street_1), timeout=10)  # server timeout time is 10sec
        location_2 = geolocator.geocode(addAvenue(street_2), timeout=10)
        
        # returns midpoint coordinates between the two locations
        # e.g. (52nd st. greenwood avenue / 52nd st. university avenue)
        return ((location_1.latitude + location_2.latitude)/2, (location_1.longitude + location_2.longitude)/2)
            
    except IndexError:
        # only single address
        location = geolocator.geocode(address, timeout=10)
        if location != None:
            return (location.latitude, location.longitude)
        
    except GeocoderTimedOut:
        # server timeout value if any (check later)
        print('Server timeout!')
        return 0
    
    except:
        # error value will point to unforseen error if any (check later)
        print('An error occurred!')
        return 999
    
    

In [11]:
# add a progress bar for the next cell (over 10K lines)
from tqdm._tqdm_notebook import tqdm_notebook

# change pandas apply to progress_apply
# tqdm adds minute overhead (~90ns; see tqdm documentation) 
tqdm_notebook.pandas(desc="Geocoding Progress!")  

In [12]:
# create batch and save files as checkpoints
batch = np.linspace(0, df['Location'].shape[0], 6, dtype=int) # each will have 2015 entries
for i in range(1, 6):
    coordinates = df['Location'][batch[i-1]: batch[i]].progress_apply(address2coordinates)
    coordinates.to_csv(('data/coordinates/coordinates_{}.csv'.format(i)))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Geocoding Progress!', max=2015), HTML(value='')))

In [171]:
# concatenate csv files and save

# use magic command to find path to files
filenames = !find -L data -name "*.csv"
coordinates = pd.concat([pd.read_csv(f, header=None) for f in filenames])
coordinates = coordinates.set_index(0)  # fix index

In [111]:
# where are errors 0 (server timeout) and 999 (other error) if any
server_timeout = np.argwhere(coordinates.values==0)[:, 0]
error_999 = np.argwhere(coordinates.values==999)[:, 0]
server_timeout

array([ 103, 3352, 3925, 4443, 6193, 8229, 9908])

In [71]:
# how many issues
print('Number of timeouts:\t{}\nNumber of 999 errors:\t{}'.format(len(server_timeout), len(error_999)))

Number of timeouts:	7
Number of 999 errors:	0


In [75]:
# redo timeouts
redo_addresses = df['Location'].values[server_timeout]
redo_addresses

'''
REMOVE:
'Various Campus Buildings' - invalid address
220 W. - lost propert Public Transit
53rd - lost cellphone
5640 S. - lost propert Public Transit

WHY:
Invalid Address or wide street range
'''

array(['Various Campus Buildings', '220 W. Garfield to 1000 E. 55th St.',
       '53rd & Kimbark to 60th & Ingleside',
       '900 E. 57th St. (Knapp Center)', '5250 S. Harper Ct.',
       '5640 S. Ellis to 56th & Cornell',
       '5525 S. Cottage Grove (Public Way)'], dtype=object)

In [216]:
# get geolocation again
redo_list = [address2coordinates(arr) for arr in redo_addresses]
redo_list

[0,
 0,
 0,
 (41.7916469, -87.6034081),
 (41.7996959, -87.5894821),
 0,
 (41.794078, -87.60576200000001)]

In [218]:
# pd.to_csv wrote data as a string
# that's why we are converting here
# easy fix will follow below
redo_list = np.array(list(map(str, redo_list)))[:, np.newaxis]
redo_list.shape

(7, 1)

In [211]:
coordinates.iloc[server_timeout] = redo_list
coordinates.iloc[server_timeout]

,1
0,
107,0
3467,0
4052,0
4593,"(41.7916469, -87.6034081)"
6449,"(41.7996959, -87.5894821)"
9010,0
11063,"(41.794078, -87.60576200000001)"


In [219]:
# change string tuple coordinates to tuple with numberic coordinates
coordinates = coordinates[1].apply(eval)  

# save fix merge
coordinates.to_csv('data/coordinates/merged_coordinates.csv')

In [276]:
#drop rows with coordinate = 0
df = df[df['Coordinates'] != 0]

## Consolidate number of Incidents

In [277]:
# number of unique incidents
df['Incident'].unique().shape

(549,)

In [299]:
# remove trailing whitespace and lowercase all
df['Incident'] = df['Incident'].apply(lambda x: x.strip().lower())
# number of unique incidents
df['Incident'].unique().shape

(506,)